In [4]:
from datetime import datetime,timedelta
import time
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout,Bidirectional
from keras.layers import LSTM
from keras import initializers

In [5]:
import pickle

In [6]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [7]:
now = datetime.now()
now.replace(microsecond=0)

# Calculate start and end limits
enddate = (now - timedelta(days=2)).replace(microsecond=0)
startdate = (now - timedelta(days=18)).replace(microsecond=0)

# url & getting api response
getter = f"https://api.twelvedata.com/time_series?apikey=e76157c75c3a42649e168c5c206e88ca&interval=5min&outputsize=5000&order=asc&start_date={startdate}&end_date={enddate}&format=JSON&symbol=BTC/INR&timezone=Asia/Kolkata"
data = requests.get(getter).json()
data_final = pd.DataFrame(data['values'])

# Data preprocessing
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data_final['close'].values.reshape(-1,1))
timeinterval = 24
prediction = 12

x_train = []
y_train = []

for i in range(timeinterval, len(scaled_data)-prediction):
    x_train.append(scaled_data[i-timeinterval:i,0])
    y_train.append(scaled_data[i+prediction,0])

x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [8]:


# Model architecture
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True,input_shape=(x_train.shape[1],1), activation='relu')))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(64, return_sequences=True, activation='tanh')))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64,activation='relu')))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy','mean_absolute_error'])

# Model training
model.fit(x_train, y_train, epochs=10, batch_size=100)

# Optionally, return a response or do further processing here



Epoch 1/10


46/46 [==============================] - 15s 92ms/step - loss: 0.0154 - accuracy: 2.1877e-04 - mean_absolute_error: 0.0888
Epoch 2/10
46/46 [==============================] - 4s 84ms/step - loss: 0.0055 - accuracy: 4.3754e-04 - mean_absolute_error: 0.0479
Epoch 3/10
46/46 [==============================] - 4s 86ms/step - loss: 0.0053 - accuracy: 4.3754e-04 - mean_absolute_error: 0.0474
Epoch 4/10
46/46 [==============================] - 4s 90ms/step - loss: 0.0049 - accuracy: 4.3754e-04 - mean_absolute_error: 0.0457
Epoch 5/10
46/46 [==============================] - 4s 90ms/step - loss: 0.0048 - accuracy: 4.3754e-04 - mean_absolute_error: 0.0451
Epoch 6/10
46/46 [==============================] - 4s 86ms/step - loss: 0.0049 - accuracy: 4.3754e-04 - mean_absolute_error: 0.0460
Epoch 7/10
46/46 [==============================] - 4s 88ms/step - loss: 0.0048 - accuracy: 4.3754e-04 - mean_absolute_error: 0.0456
Epoch 8/10
46/46 [==============================] - 4s 83ms/step

In [9]:
# Save the model using pi
with open('new_model.pkl','wb') as f:
  pickle.dump(model,f)